In [73]:
!pip install bertopic

In [1]:
!pip3 install h5py 

In [75]:
!pip install typing-extensions 

In [76]:
!pip install wheel 

In [77]:
!pip install --upgrade joblib==1.1.0

In [78]:
import numpy
numpy.version.version

'1.20.0'

In [79]:
from bertopic import BERTopic

In [80]:
import pandas as pd
import numpy as np

In [81]:
# Text preprocessiong
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [82]:
# Topic model
from bertopic import BERTopic

In [83]:
# Dimension reduction
from umap import UMAP

In [84]:
# Read in data
usptodata = pd.read_csv('U.S. Patents.csv')

In [85]:
usptodataset=usptodata[["grant_id","claims_text","abstract"]]

In [86]:
usptodataset

,grant_id,claims_text,abstract
0,USD0864516,"The ornamental design for a thin food cluster,...",NaN
1,USD0864517,I claim the ornamental design for a chocolate ...,NaN
2,USD0864518,The ornamental design for a golf glove with ve...,NaN
3,USD0864519,"We claim the ornamental design for a glove, as...",NaN
4,USD0864520,The ornamental design of a wearable towel with...,NaN
...,...,...,...
7971,US10462947,"1. A component mounting machine comprising:,a ...",Provided are a first component holding tool op...
7972,US10462948,1. A mounting deviation correction apparatus w...,In a case in which mounting deviation is occur...
7973,US10462949,1. A reel holding device configured for use wi...,A reel holding device is provided. The device ...
7974,US10462950,1. An electronic component bonding device for ...,An electronic component bonding device include...


In [87]:
usptodataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7976 entries, 0 to 7975
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   grant_id     7976 non-null   object
 1   claims_text  7976 non-null   object
 2   abstract     7013 non-null   object
dtypes: object(3)
memory usage: 187.1+ KB


In [88]:
# Remove stopwords
new_stopwords = ["shown", "described", "design", "ornamental", "describe", "described", "described."]
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(new_stopwords)
print(f'There are {len(stopwords)} default stopwords. They are {stopwords}')

There are 186 default stopwords. They are ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'no

In [89]:
# Remove stopwords
usptodataset['claims_text_without_stopwords'] = usptodataset['claims_text'].apply(lambda x: ' '.join([w for w in x.split() if w.lower() not in stopwords]))

# Lemmatization
usptodataset['claims_text_lemmatized'] = usptodataset['claims_text_without_stopwords'].apply(lambda x: ' '.join([wn.lemmatize(w) for w in x.split() if w not in stopwords]))

# Take a look at the data
#usptodataset.head()

In [90]:
# Punctuation
import string
punct = string.punctuation

In [91]:
def remove_punct(x):
    return x.translate(str.maketrans("","",punct))
usptodataset['claims_text_lemmatized'] = usptodataset['claims_text_lemmatized'].apply(lambda x: remove_punct(x))

In [92]:
# remove of word frequency
from collections import Counter
cnt = Counter()

for text in usptodataset['claims_text_lemmatized'].values:
    for word in text.split():
        cnt[word]+=1
cnt.most_common(10)        

[('first', 108927),
 ('wherein', 107314),
 ('claim', 99957),
 ('second', 90457),
 ('one', 66588),
 ('1', 51174),
 ('device', 48384),
 ('least', 45491),
 ('said', 42920),
 ('method', 41556)]

In [93]:
freq_words = set([w for (w,wc) in cnt.most_common(10)])
freq_words

{'1',
 'claim',
 'device',
 'first',
 'least',
 'method',
 'one',
 'said',
 'second',
 'wherein'}

In [94]:
import string
def remove_freq(x):
    
    return " ".join([word for word in str(x).split() if word not in freq_words])


usptodataset['claims_text_without_freq']= usptodataset['claims_text_lemmatized'].apply(lambda x: remove_freq(x))

In [95]:
usptodataset.iloc[1000]

grant_id                                                                US10455913
claims_text                      1. A luggage, comprising a luggage body, a lug...
abstract                         A luggage includes a luggage body, a luggage w...
claims_text_without_stopwords    1. luggage, comprising luggage body, luggage w...
claims_text_lemmatized           1 luggage comprising luggage body luggage whee...
claims_text_without_freq         luggage comprising luggage body luggage wheel ...
Name: 1000, dtype: object

In [97]:
# Initiate UMAP
umap_model = UMAP(#n_neighbors=15, 
                  #n_components=5, 
                  min_dist=0.0, 
                  metric='cosine', 
                  random_state=100)

# Initiate BERTopic
topic_model = BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True)

# Run BERTopic model
topics, probabilities = topic_model.fit_transform(usptodataset['claims_text_without_freq'])

In [98]:
# Get the list of topics
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,2719,-1_data_system_user_plurality
1,0,302,0_image_pixel_camera_object
2,1,198,1_resource_station_transmission_wireless
3,2,176,2_semiconductor_layer_dielectric_gate
4,3,155,3_turbine_fuel_gas_engine
...,...,...,...
134,133,12,133_wheel_automotive_chocolate_shown
135,134,12,134_display_cursor_displayed_region
136,135,11,135_packaging_lipstick_therapy_vapor
137,136,10,136_electrode_capacitor_resistance_termination


In [99]:
topic_model.get_topic_info(100)

,Topic,Count,Name
0,100,19,100_chair_stool_desk_merrygoround


In [100]:
# Visualize connections between topics using hierachical clustering
topic_model.visualize_hierarchy(top_n_topics=50)

In [101]:
# Visualize probability distribution
topic_model.visualize_distribution(topic_model.probabilities_[1000], min_probability=0.015)

In [103]:
topic_model.get_topic(topic=26)

[('drawer', 0.020932541248935514),
 ('member', 0.02033840764505826),
 ('slide', 0.01858052927841247),
 ('framework', 0.01738229018011543),
 ('position', 0.0160966350834255),
 ('actuator', 0.015816879329282937),
 ('exercise', 0.01499651008421755),
 ('plate', 0.014793886588422235),
 ('shaft', 0.014292057860362162),
 ('holding', 0.014062670983497503)]

In [104]:
# Check the content for the first review
usptodataset['claims_text'][1000]

'1. A luggage, comprising a luggage body, a luggage wheel mounted to each of two opposite lateral sides of said luggage body and a support mechanism mounted to each of the said two opposite lateral sides of said luggage body corresponding to one respective said luggage wheel, wherein:,said support mechanism comprises a locating shaft, a support member and a position-limiting member, said locating shaft being affixed to said luggage body and comprising a first locating groove and a second locating groove located on the periphery thereof and a stop rib disposed between said first locating groove and said second locating groove, said support member comprising a base block, a pivot hole located on said base block and pivotally coupled to said locating shaft, a support portion located on one end of said base block and an actuation portion located on an opposite end of said base block, said position-limiting member being mounted in said base block of said support member and transversely mova

In [105]:
# Visualize intertopic distance
topic_model.visualize_topics()

In [106]:
# New data for the review
new_review = " A luggage, comprising a luggage body, a luggage wheel mounted to each of two opposite lateral sides of said luggage body and a support mechanism mounted to each of the said two opposite lateral sides of said luggage body corresponding to one respective said luggage wheel, wherein: said support mechanism comprises a locating shaft, a support member and a position-limiting member, said locating shaft being affixed to said luggage body and comprising a first locating groove and a second locating groove located on the periphery thereof and a stop rib disposed between said first locating groove and said second locating groove, said support member comprising a base block, a pivot hole located on said base block and pivotally coupled to said locating shaft, a support portion located on one end of said base block and an actuation portion located on an opposite end of said base block, said position-limiting member being mounted in said base block of said support member and transversely movable in and out of said pivot hole such that when said luggage body is tilted for movement by said luggage wheels, said position-limiting member is engaged in said second locating groove to lock said support member in a position where said actuation portion protrudes over a bottom side of said luggage body; when said luggage body is shifted from said tilted position to an erected position, said actuation portion is forced by the floor to bias said support member, causing movement of said position-limiting member out of said second locating groove over said stop rib into said first positioning groove and simultaneously forcing said support portion into abutment against the floor where the distance between said support portion and a front side of said luggage body is larger than the distance between the axis of said luggage wheels and the said front side of said luggage body."
# Find topics
num_of_topics = 3
similar_topics, similarity = topic_model.find_topics(new_review, top_n=num_of_topics); 

# Print results
print(f'The top {num_of_topics} similar topics are {similar_topics}, and the similarities are {np.round(similarity,2)}')

The top 3 similar topics are [60, 133, 26], and the similarities are [0.34 0.32 0.31]


In [107]:
# Print the top keywords for the top similar topics
for i in range(num_of_topics):
    print(f'The top keywords for topic {similar_topics[i]} are:')
    print(topic_model.get_topic(similar_topics[i]))

The top keywords for topic 60 are:
[('frame', 0.027393379377124105), ('support', 0.021968630582505345), ('seating', 0.021945037676906566), ('snare', 0.02185620036228634), ('wheel', 0.021614715227951487), ('work', 0.020499728382991654), ('counterweight', 0.0203802511423538), ('horizontal', 0.019770465462337586), ('member', 0.01940376529855519), ('position', 0.019108153538811345)]
The top keywords for topic 133 are:
[('wheel', 0.8353264110858794), ('automotive', 0.5363186398286548), ('chocolate', 0.3794003523078551), ('shown', 0.27673164087508567), ('wheeled', 0.26021433972443364), ('luggage', 0.25736656378938194), ('wheelchair', 0.24883786451938922), ('sport', 0.24571005756789313), ('spindle', 0.1837972857694898), ('vehicle', 0.17524323873603703)]
The top keywords for topic 26 are:
[('drawer', 0.020932541248935514), ('member', 0.02033840764505826), ('slide', 0.01858052927841247), ('framework', 0.01738229018011543), ('position', 0.0160966350834255), ('actuator', 0.015816879329282937), ('

In [110]:
# Visualize top topic keywords
topic_model.visualize_barchart(similar_topics)